In [1]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph.message import MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool


llm = init_chat_model("openai:gpt-4o-mini")


In [2]:
class State(MessagesState):
    custom_stuff: str


graph_builder = StateGraph(State)

In [3]:
@tool
def get_weather(city:str):
    """Gets weather in city"""
    return f"The weather in {city} is sunny."

llm_with_tools = llm.bind_tools(tools=[get_weather])

def chatbot(state: State):
    response = llm.invoke(state["messages"])
    return {
        "messages": [response]
    }

tool_node = ToolNode(
    tools=[
        get_weather
    ],
)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")



In [4]:
graph = graph_builder.compile()

graph.invoke(
    {
        "messages": [
        {"role": "user", "content": "what is the weather in Barcelona?"}
        ]
    }
)

{'messages': [HumanMessage(content='what is the weather in Barcelona?', additional_kwargs={}, response_metadata={}, id='25cac772-2f46-4a2d-b2ea-70c3000fd243'),
  AIMessage(content="I can't provide real-time weather updates. For the most accurate and up-to-date weather information in Barcelona, I recommend checking a reliable weather website or app.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 14, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DBQIOCPCNJze9VEyy7bWlsz2Yz4wK', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--019c7c7c-7ede-7321-992b-7e26afa4f0e8-0', usage_metadata={'input_tokens': 14, 'output_tokens': 31